In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


/home/fred/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import random
from collections import defaultdict

def load_data(data_file):
    triples = []
    with open(data_file, 'r') as f:
        for line in f:
            triple = line.strip()  
            triples.append(triple)
    return triples

def generate_negative_samples(triples, sampling_strategy='random_plausible'):
    all_entities = set()
    all_relations = set()
    # relation_counts = defaultdict(lambda: defaultdict(int)) 

    for triple in triples:
        triple = triple.split(" ")
        subject, relation, object =  triple[1], triple[3], triple[5]
        all_entities.add(subject)
        all_entities.add(object)
        all_relations.add(relation)
        # relation_counts[relation][subject] += 1 
        # relation_counts[relation][object] += 1

    negative_samples = []
    for triple in triples:
        triple = triple.split(" ")
        subject, original_relation, original_object = triple[1], triple[3], triple[5]

        neg_sample = 5
        while neg_sample > 0:
            if sampling_strategy == 'random_plausible':
                
                negative_object = random.choice(list(all_entities))
                negative_relation = random.choice(list(all_relations))
                while negative_object == original_object:  # Ensure we get a different object
                    negative_object = random.choice(list(all_entities))
                while negative_relation == original_relation:
                    negative_relation = random.choice(list(all_relations))


            negative_sample = f"[CLS] {subject} [SEP1] {negative_relation} [SEP2] {negative_object} [END]" 
            negative_samples.append(negative_sample)
            neg_sample -= 1

    return negative_samples

# Example Usage 
data_file = './data/WN18RR_torch/sequences/train_seq.txt'
triples = load_data(data_file)
negative_samples = generate_negative_samples(triples, sampling_strategy='random_plausible')

print("Original Triples:")
print(triples[:5])  # Print a few original samples

print("Negative Samples:")
print(negative_samples[:5])  # Print a few negative samples 


# Assuming you have 'triples' (true samples) and 'negative_samples'
data = triples + negative_samples
labels = [1] * len(triples) + [0] * len(negative_samples) 

In [26]:
# # save generated data
# import pickle
# with open("./data/WN18RR_torch/with_5_negative_samples_data.pickle", "wb") as fp:   #Pickling
#     pickle.dump(data, fp)

# with open("./data/WN18RR_torch/with_5_negative_samples_labels.pickle", "wb") as fp:   #Pickling
#     pickle.dump(labels, fp)

In [3]:
# Load data
import pickle
with open("./data/WN18RR_torch/with_5_negative_samples_data.pickle", "rb") as fp:   # Unpickling
    data = pickle.load(fp)

with open("./data/WN18RR_torch/with_5_negative_samples_labels.pickle", "rb") as fp:   # Unpickling
    labels = pickle.load(fp)


In [2]:
def load_data(data_file):
    triples = []
    with open(data_file, 'r') as f:
        for line in f:
            triple = line.strip()  
            triples.append(triple)
    return triples

# Example Usage 
data_file = './data/WN18RR_torch/sequences/train_seq.txt'
triples = load_data(data_file)
# negative_samples = generate_negative_samples(triples, sampling_strategy='random_plausible')


print("Original Triples:")
print(triples[:5])  # Print a few original samples

# print("Negative Samples:")
# print(negative_samples[:5])  # Print a few negative samples 

all_entities = set()
all_relations = set()
# relation_counts = defaultdict(lambda: defaultdict(int)) 

for triple in triples:
    triple = triple.split(" ")
    subject, relation, object =  triple[1], triple[3], triple[5]
    all_entities.add(int(subject))
    all_entities.add(int(object))
    all_relations.add(int(relation))

all_entities = list(all_entities)
all_relations = list(all_relations)
print()
print(all_entities[:10])

Original Triples:
['[CLS] 260881 [SEP1] 0 [SEP2] 260622 [END]', '[CLS] 1332730 [SEP1] 1 [SEP2] 3122748 [END]', '[CLS] 6066555 [SEP1] 1 [SEP2] 645415 [END]', '[CLS] 9322930 [SEP1] 2 [SEP2] 9360122 [END]', '[CLS] 7193596 [SEP1] 1 [SEP2] 784342 [END]']

[9830400, 6946823, 524299, 393227, 14942223, 655378, 131090, 13631512, 4456472, 786458]


In [6]:
print(len(triples))
# print(len(negative_samples))

86835


In [7]:
SPECIAL_TOKENS = ['[CLS]', '[SEP1]', '[SEP2]', '[END]']
str_to_token_int_map = {'[CLS]':100, '[SEP1]':200, '[SEP2]':300, '[END]':400} # Special tokens first, ...

def create_vocabulary(data_file):
    """Creates a vocabulary (index mapping) from your dataset file."""
    all_ids = set()
    with open(data_file, 'r') as f:
        for line in f:
            triple = line.strip().split(" ")
            subject, relation, object = triple[1], triple[3], triple[5]
            ids = [int(subject), int(relation),int(object)]
            all_ids.update(ids)  

    # ... then numeric IDs
    # str_to_token_int_map.update({str(token): token for token in all_ids})
    str_to_token_int_map.update({str(token): index+1000 for index, token in enumerate(all_ids)})
    return str_to_token_int_map

def tokenize_triple(triple_str, str_to_token_int_map=str_to_token_int_map):
    """Tokenizes a single triple using the provided index mapping."""
    tokens = triple_str.strip().split(" ")
    numerical_ids = [str_to_token_int_map[token] for token in tokens]
    return numerical_ids

# Example Usage
data_file = './data/WN18RR_torch/sequences/train_seq.txt'
index_map = create_vocabulary(data_file)

triple_str = '[CLS] 13266892 [SEP1] 1 [SEP2] 8107499 [END]'
tokenized_triple = tokenize_triple(triple_str, index_map)
print(tokenized_triple)  # Output: A list of numerical tokens 


[100, 9917, 200, 1001, 300, 35799, 400]


In [ ]:
[100, 13266892, 200, 1, 300, 8107499, 400]

In [33]:
index_map['13266892']

9917

In [4]:
all_ids = set()
with open(data_file, 'r') as f:
    for line in f:
        triple = line.strip().split(" ")
        subject, relation, object = triple[1], triple[3], triple[5]
        ids = [int(subject), int(relation),int(object)]
        all_ids.update(ids)
len(all_ids)
print(all_ids)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 524299, 6946823, 393227, 14942223, 655378, 131090, 13631512, 4456472, 786458, 5111835, 8388636, 2359324, 4325409, 14286885, 12320806, 3932203, 1572910, 3014705, 2752567, 14155834, 1179707, 7340094, 2490430, 11927616, 5636171, 12451915, 14549070, 1835087, 12189779, 1966168, 4849759, 1835103, 11534434, 2752615, 13238375, 262249, 5505131, 917614, 1572978, 3145843, 2228341, 1966204, 6684798, 10223744, 7209089, 12714114, 2228355, 4587648, 9961605, 4063373, 1048718, 13500557, 9306257, 7733394, 917651, 393369, 13893786, 14024882, 3408054, 2752695, 14418103, 4980920, 9437369, 11927740, 2097341, 8913085, 3670208, 655555, 14680261, 12058822, 13631687, 1441993, 1573074, 10354898, 6422740, 12583126, 6947032, 1179865, 7340249, 9044190, 9830629, 8519916, 524530, 1310964, 917759, 11796744, 14287113, 2490634, 9961739, 1835276, 917772, 9437454, 12714254, 1966352, 1835280, 2621706, 13762836, 7602454, 2621721, 12189987, 1966377, 1704236, 5636402, 4981044, 12321077, 4849

In [12]:
# tokenized_data = torch.tensor(list(map(tokenize_triple, data)), dtype=torch.int32).to(device)
tokenized_data.shape

torch.Size([521010, 7])

In [15]:
torch.tensor(labels)

tensor([1, 1, 1,  ..., 0, 0, 0])

In [5]:
%load_ext autoreload
%autoreload 2
from transformer import EncoderOnlyTransformer

In [6]:
def tokenizer(batch_data):
    return torch.tensor(list(map(tokenize_triple, batch_data)), dtype=torch.int32).to(device)

In [6]:

from torch.utils.data import DataLoader, TensorDataset  # For creating Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
del EncoderOnlyTransformer
from transformer import EncoderOnlyTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

num_epochs = 10

src_vocab_size = tgt_vocab_size = len(index_map) + 2000  # = 40574 + 2000
d_model= 256
num_heads= 8
num_layers= 6
d_ff = d_model*4  # (approximately 4 times d_model)
max_seq_length= 7
dropout= 0.1
model = EncoderOnlyTransformer(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

tokenized_data = torch.tensor(list(map(tokenize_triple, data)), dtype=torch.int32).to(device)

dataset = TensorDataset(tokenized_data, torch.tensor(labels).to(device))  # Create a PyTorch Dataset
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)  
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
model.train()

for epoch in range(num_epochs):
    cnt = 0
    for batch_data, batch_labels in tqdm(dataloader):

        output = model(batch_data)  # Calculate predictions
        loss = criterion(output.squeeze(), batch_labels.float())  # Calculate loss using labels

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        cnt += 1
    print(f"Epoch: {epoch+1}, loss: {loss.item()}") 


device: cuda


NameError: name 'data' is not defined

In [109]:
index_map[41186]

KeyError: 41186

In [38]:
data[0]

'[CLS] 260881 [SEP1] 0 [SEP2] 260622 [END]'

In [32]:
def predict_entity(subject, relation, model=model, candidate_objects=all_entities):
    scores = []
    model.eval()
    for obj in candidate_objects:
        input_seq = [f"[CLS] {subject} [SEP1] {relation} [SEP2] {obj} [END]"]
        input_data = tokenizer(input_seq)  
        output_score = model.forward(input_data)
        output = torch.sigmoid(output_score).squeeze()
        scores.append(output)

    best_index = scores.index(max(scores))
    return candidate_objects[best_index]

In [33]:
predict_entity(260881, 0)

In [10]:

# predict_entity(260881, 0)
model.eval()
scores=[]
for obj in tqdm(all_entities):
    input_seq = [f"[CLS] {260881} [SEP1] {0} [SEP2] {obj} [END]"]
    input_data = tokenizer(input_seq)  
    output_score = model.forward(input_data)
    output = torch.sigmoid(output_score).squeeze()
    scores.append(output)
best_index = scores.index(max(scores))
print(best_index)

 22%|██▏       | 9100/40559 [00:39<02:16, 229.99it/s]


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.91 GiB total capacity; 9.64 GiB already allocated; 32.19 MiB free; 9.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF